In [1]:
import pandas as pd
import numpy as np

In [2]:
test = pd.read_table('products_sentiment_test.tsv')
test.head()

,Id,text
0,0,"so , why the small digital elph , rather than ..."
1,1,3/4 way through the first disk we played on it...
2,2,better for the zen micro is outlook compatibil...
3,3,6 . play gameboy color games on it with goboy .
4,4,"likewise , i 've heard norton 2004 professiona..."


In [3]:
train = pd.read_table('products_sentiment_train.tsv')

In [4]:
train.rename(columns={'2 . take around 10,000 640x480 pictures .':'text',
                     '1':'y'}, inplace=True)

In [5]:
train.head()

,text,y
0,i downloaded a trial version of computer assoc...,1
1,the wrt54g plus the hga7t is a perfect solutio...,1
2,i dont especially like how music files are uns...,0
3,i was using the cheapie pail ... and it worked...,1
4,"you can manage your profile , change the contr...",1


In [6]:
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\preductor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
stop_words_1 = set(nltk.corpus.stopwords.words('english'))
stop_words_2 = ENGLISH_STOP_WORDS

In [8]:
def write_ans(preds):
    submission = pd.read_csv('products_sentiment_sample_submission.csv')
    submission['y'] = preds
    submission.to_csv(r'C:\Users\preductor\Documents\MachineLearning\6_course\Semantic analysis\week_3\submission.csv', index=False)

## ----------------------------------------------------------------------------------

### LogisticRegression --------------------------------------------------------------------------------

<font color='green'>Задумка была следующая: взять Pipeline из векторайзера и простого линейного классификатора(как оказалось LogisticRegression выдает лучшие результаты) и построить GridSearchCV с перебором разных комбинаций параметров. И уже векторайзер с лучшими результатами запихнуть в более продвинутый классификатор</font>

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

C:\Users\preductor\AppData\Local\conda\conda\envs\py3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [52]:
clf_1 = Pipeline(steps=[('vec',CountVectorizer()), ('classifier',LogisticRegression())])
clf_2 = Pipeline(steps=[('vec',TfidfVectorizer()), ('classifier',LogisticRegression())])

In [53]:
param_grid = {'vec__min_df' : [1,2,3],
          'vec__max_df' : [50,100,200,300],
          'vec__stop_words' : [stop_words_1, stop_words_2],
          'vec__ngram_range' : [(1,3),(1,5),(1,6)],
          'vec__analyzer' : ['char','word','char_wb']}

In [54]:
gscv = GridSearchCV(clf_1, param_grid=param_grid, scoring='roc_auc', cv=3, verbose=False)

In [55]:
gscv.fit(X=train.text, y=train.y)
gscv.best_score_

0.8494700156877127

In [56]:
gscv.best_estimator_

Pipeline(memory=None,
     steps=[('vec', CountVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=100, max_features=None, min_df=1,
        ngram_range=(1, 6), preprocessor=None,
        stop_words={'off', 'out...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [70]:
preds = gscv.best_estimator_.predict(test.text)

In [71]:
write_ans(preds)

### Xgboost --------------------------------------------------------------------------------------

<font color='green'>Но как бы я не экспериментировал с Xgboost все та же логистическая регрессия выдавала лучшие результаты. Вобщем после ряда неудачных попыток улучшения, я решил оставить как есть, так как Simple benchmark пройден</font>

In [25]:
import xgboost

C:\Users\preductor\AppData\Local\conda\conda\envs\py3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [72]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(train.text, train.y, test_size=0.25, stratify=train.y)

In [73]:
vec = CountVectorizer(min_df=1, max_df=100, ngram_range=(1,6),
        analyzer='char')
data = vec.fit_transform(train.text, train.y)

In [ ]:
clf = xgboost.XGBClassifier(max_depth=3, n_estimators=500)
clf.fit(vec.transform(train_X).toarray(), train_y, eval_set=[(vec.transform(test_X).toarray(), test_y)],
        eval_metric='auc', early_stopping_rounds=100)

In [42]:
from sklearn import metrics
y_pre = clf.predict(vec.transform(train.text).toarray())
y_pro = clf.predict_proba(vec.transform(train.text).toarray())[:, 1]
print("AUC Score : %f" % metrics.roc_auc_score(train.y, y_pro))
print("Accuracy : %.4g" % metrics.accuracy_score(train.y, y_pre))

C:\Users\preductor\AppData\Local\conda\conda\envs\py3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


AUC Score : 0.959811
Accuracy : 0.8934


In [43]:
write_ans(clf.predict(vec.transform(test.text).toarray()))

C:\Users\preductor\AppData\Local\conda\conda\envs\py3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
